In [2]:
import argparse
import sys

from copy import deepcopy

import os

from dgl import load_graphs, save_graphs
from rdkit import Chem
from rdkit.Chem import RemoveHs
from rdkit.Geometry import Point3D
from tqdm import tqdm

In [3]:
from commons.geometry_utils import rigid_transform_Kabsch_3D, get_torsions, get_dihedral_vonMises, apply_changes
from commons.logger import Logger
from commons.process_mols import read_molecule, get_lig_graph_revised, \
    get_rec_graph, get_geometry_graph, get_geometry_graph_ring, \
    get_receptor_inference

In [4]:
from train import load_model

from datasets.pdbbind import PDBBind

from commons.utils import seed_all, read_strings_from_txt

import yaml

In [5]:
from datasets.custom_collate import*  # do not remove
from models import *  # do not remove
from torch.nn import *  # do not remove
from torch.optim import *  # do not remove
from commons.losses import *  # do not remove
from torch.optim.lr_scheduler import *  # do not remove

from torch.utils.data import DataLoader


In [6]:
from trainer.metrics import Rsquared, MeanPredictorLoss, MAE, PearsonR, RMSD, RMSDfraction, CentroidDist, \
    CentroidDistFraction, RMSDmedian, CentroidDistMedian

# turn on for debugging C code like Segmentation Faults
import faulthandler

In [7]:
from Bio.PDB import *


In [8]:
from joblib import Parallel, delayed, cpu_count
import torch
import dgl
from biopandas.pdb import PandasPdb
from joblib.externals.loky import get_reusable_executor

In [9]:
import torch.nn.functional as F

from commons.geometry_utils import random_rotation_translation, rigid_transform_Kabsch_3D_torch
from commons.process_mols import get_rdkit_coords, get_receptor, get_pocket_coords, \
    read_molecule, get_rec_graph, get_lig_graph_revised, get_receptor_atom_subgraph, get_lig_structure_graph, \
    get_geometry_graph, get_lig_graph_multiple_conformer, get_geometry_graph_ring
from commons.utils import pmap_multi, read_strings_from_txt, log

In [10]:
from commons.process_mols import get_rdkit_coords, get_receptor, get_pocket_coords, \
    read_molecule, get_rec_graph, get_lig_graph_revised, get_receptor_atom_subgraph, get_lig_structure_graph, \
    get_geometry_graph, get_lig_graph_multiple_conformer, get_geometry_graph_ring

In [11]:
pip install nglview

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install py3Dmol

Note: you may need to restart the kernel to use updated packages.


In [32]:
import nglview
import py3Dmol

In [14]:
from rdkit.Chem import AllChem
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from rdkit.Chem import PandasTools

In [15]:
from datasets.pdbbind import PDBBind

In [33]:
import argparse
import sys

from copy import deepcopy
import os
from dgl import load_graphs, save_graphs
from rdkit import Chem
from rdkit.Chem import RemoveHs
from rdkit.Geometry import Point3D
from tqdm import tqdm
from datasets.pdbbind import PDBBind
from Bio.PDB import *
from joblib import Parallel, delayed, cpu_count
import torch
import dgl
from biopandas.pdb import PandasPdb
from joblib.externals.loky import get_reusable_executor
import nglview as nv
import py3Dmol
filepath=os.path.join("gs","l10gs_protein_processed.pdb")
#view1=nv.NGLWidget()
view1= nv.show_file(filepath)
view1
view1.clear_representations()
view1.add_representation("hyperball","ligand", color="blue", opacity=0.9)
view1.add_representation("cartoon","protein", color="red", opacity=0.9)
view1.center("ligand")

In [16]:
os.listdir('test_visualization')

['4wpn_ligand.mol2',
 '4wpn_ligand.sdf',
 'firstnmage.o.png',
 'l4wpn_ligand.sdf',
 'l4wpn_protein_processed.pdb',
 'lig_equibind_corrected.sdf',
 'lig_equibind_corrected1.csv',
 'protein_processed.pdb']

In [17]:
pdbl= PDBList()

In [18]:
parser= MMCIFParser()

In [19]:
structure=parser.get_structure('4wpn', "wp\l4wpn.cif")

In [20]:
def cleandir(obj):
    print(",".join([a for a in dir(obj)if not a.startswith("_")]))
    
cleandir(structure); 

add,atom_to_internal_coordinates,center_of_mass,child_dict,child_list,copy,detach_child,detach_parent,full_id,get_atoms,get_chains,get_full_id,get_id,get_iterator,get_level,get_list,get_models,get_parent,get_residues,has_id,header,id,insert,internal_to_atom_coordinates,level,parent,set_parent,transform,xtra


In [28]:
view1= nglview.show_biopython(structure, gui=True)

In [29]:
pip list


Package                       Version
----------------------------- -----------
absl-py                       1.3.0
aiohttp                       3.8.3
aiosignal                     1.3.1
anyio                         3.6.2
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.1.0
async-timeout                 4.0.2
asynctest                     0.13.0
attrs                         22.1.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.11.1
biopandas                     0.4.1
biopython                     1.79
bleach                        5.0.1
blinker                       1.5
brotlipy                      0.7.0
cachetools                    5.2.0
certifi                       2022.12.7
cffi                          1.15.1
charset-normalizer            2.1.1
click                         8.1.3
cloudpickle                   2.2.0
colorama                      0.4.6
cryptography 

In [31]:
view1

AttributeError: 'super' object has no attribute '_ipython_display_'

NGLWidget()

In [22]:
# Extracting coordinates of Ligand

In [24]:
pdb_parser=PDBParser()


In [41]:
my_sdf_file ='test_visualization\l4wpn_ligand.sdf' # have extracted smiles from sdf file

frame = PandasTools.LoadSDF(my_sdf_file,
                            smilesName='SMILES',
                            molColName='Molecule',
                            includeFingerprints=False)


In [42]:
Structure_Protein=pdb_parser.get_structure('4wpn', "test_visualization\l4wpn_protein_processed.pdb")

In [43]:
cleandir(Structure_ligand); 

add,atom_to_internal_coordinates,center_of_mass,child_dict,child_list,copy,detach_child,detach_parent,full_id,get_atoms,get_chains,get_full_id,get_id,get_iterator,get_level,get_list,get_models,get_parent,get_residues,has_id,header,id,insert,internal_to_atom_coordinates,level,parent,set_parent,transform,xtra


In [45]:
import re
from pprint import pprint

file = open('test_visualization\l4wpn_ligand.sdf', mode="r")
content = file.read()
file.close()
match = re.search(r"^ {3,}-?\d.*(?:\r?\n {3,}-?\d.*)*", content, re.M)
datasplit = []

if match:
    for line in match.group().splitlines():
        datasplit.append([part for part in line.split()][:4])

pprint(datasplit)

[['38.4730', '19.5130', '16.4480', 'C'],
 ['37.5260', '19.0490', '15.4080', 'C'],
 ['37.1920', '19.9130', '14.2860', 'C'],
 ['36.1230', '18.0200', '14.1210', 'C'],
 ['36.8550', '17.9070', '15.2610', 'N'],
 ['36.8920', '16.7580', '16.1700', 'C'],
 ['38.3250', '16.3160', '16.4660', 'C'],
 ['37.4240', '13.9730', '16.8310', 'C'],
 ['39.8970', '14.4050', '16.8460', 'C'],
 ['35.2060', '16.9480', '13.5670', 'C'],
 ['35.0050', '14.4920', '13.8180', 'C'],
 ['34.9340', '13.5940', '12.5860', 'C'],
 ['36.8930', '14.6000', '11.5840', 'C'],
 ['37.0820', '15.5650', '12.7510', 'C'],
 ['36.5120', '12.1230', '11.3000', 'C'],
 ['39.9470', '21.3380', '17.2610', 'C'],
 ['38.7860', '18.8060', '17.4280', 'O'],
 ['39.0030', '20.8100', '16.2710', 'N'],
 ['38.6320', '21.5990', '15.1460', 'C'],
 ['39.1280', '22.7390', '15.0270', 'O'],
 ['37.7200', '21.1340', '14.1540', 'N'],
 ['37.3690', '21.9870', '13.0150', 'C'],
 ['36.3220', '19.2320', '13.5190', 'N'],
 ['38.5530', '14.8210', '16.2550', 'C'],
 ['35.8750', '15

In [50]:
os.path.isdir('test_visualization')

True

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../data/PDBBind'

In [76]:
#rec_path = os.path.join('test_visualization', name, f'{name}_protein_processed.pdb')
rec_path = os.path.join('test_visualization', 'l4wpn_protein_processed.pdb')
lig = read_molecule(os.path.join('test_visualization', 'l4wpn_ligand.sdf'), sanitize=True, remove_hs=False)

In [106]:
get_receptor(rec_path, lig, 10);  #I need to know the value of the cutoff-- last parameter as it was mentioned in one of .py
#files

In [115]:
def process(self):
        log(f'Processing complexes from [{self.complex_names_path}] and saving it to [{self.processed_dir}]')

        complex_names = read_strings_from_txt(self.complex_names_path)
        if self.dataset_size != None:
            complex_names = complex_names[:self.dataset_size]
        if (self.remove_h or self.only_polar_hydrogens) and '4acu' in complex_names:
            complex_names.remove('4acu')  # in this complex's ligand the hydrogens cannot be removed
        log(f'Loading {len(complex_names)} complexes.')
        global ligs
        rec_paths
        ligs = []
        to_remove = []
        for name in tqdm(complex_names, desc='loading ligands'):
            if self.bsp_ligands:
                lig = read_molecule(os.path.join(self.bsp_dir, name, f'Lig_native.pdb'), sanitize=True, remove_hs=self.remove_h)
                if lig == None:
                    to_remove.append(name)
                    continue
            else:
                lig = read_molecule(os.path.join(self.pdbbind_dir, name, f'{name}_ligand.sdf'), sanitize=True,
                                    remove_hs=self.remove_h)
                if lig == None:  # read mol2 file if sdf file cannot be sanitized
                    lig = read_molecule(os.path.join(self.pdbbind_dir, name, f'{name}_ligand.mol2'), sanitize=True,
                                        remove_hs=self.remove_h)
            if self.only_polar_hydrogens:
                for atom in lig.GetAtoms():
                    if atom.GetAtomicNum() == 1 and [x.GetAtomicNum() for x in atom.GetNeighbors()] == [6]:
                        atom.SetAtomicNum(0)
                lig = Chem.DeleteSubstructs(lig, Chem.MolFromSmarts('[#0]'))
                Chem.SanitizeMol(lig)
            ligs.append(lig)
        for name in to_remove:
            complex_names.remove(name)

        if self.bsp_proteins:
            rec_paths = [os.path.join(self.bsp_dir, name, f'Rec.pdb') for name in complex_names]
        else:
            rec_paths = [os.path.join(self.pdbbind_dir, name, f'{name}_protein_processed.pdb') for name in
                         complex_names]

        if not os.path.exists(self.processed_dir):
            os.mkdir(self.processed_dir)

        if not os.path.exists(os.path.join(self.processed_dir, 'rec_graphs.pt')) or not os.path.exists(os.path.join(self.processed_dir, 'pocket_and_rec_coords.pt')) or (not os.path.exists(os.path.join(self.processed_dir, self.rec_subgraph_path)) and self.rec_subgraph):
            log('Get receptors, filter chains, and get its coordinates')
            receptor_representatives = pmap_multi(get_receptor, zip(rec_paths, ligs), n_jobs=self.n_jobs, cutoff=self.chain_radius, desc='Get receptors')
            recs, recs_coords, c_alpha_coords, n_coords, c_coords = map(list, zip(*receptor_representatives))
            # rec coords is a list with n_residues many arrays of shape: [n_atoms_in_residue, 3]

        return recs, recs_coords, c_alpha_coords, n_coords, c_coords

In [117]:
process('self')

AttributeError: 'str' object has no attribute 'complex_names_path'

In [114]:
ligs

#receptor_representatives = pmap_multi(get_receptor, zip(rec_paths, ligs), n_jobs=self.n_jobs, cutoff=self.chain_radius,
#desc='Get receptors')
#recs, recs_coords, c_alpha_coords, n_coords, c_coords = map(list, zip(*receptor_representatives))
 # rec coords is a list with n_residues many arrays of shape: [n_atoms_in_residue, 3]


NameError: name 'rec_paths' is not defined

In [ ]:
def get_calpha_graph(rec, c_alpha_coords, n_coords, c_coords, cutoff=20, max_neighbor=None):
    ################## Extract 3D coordinates and n_i,u_i,v_i vectors of representative residues ################
    residue_representatives_loc_list = []
    n_i_list = []
    u_i_list = []
    v_i_list = []
    for i, residue in enumerate(rec.get_residues()):
        n_coord = n_coords[i]
        c_alpha_coord = c_alpha_coords[i]
        c_coord = c_coords[i]
        u_i = (n_coord - c_alpha_coord) / np.linalg.norm(n_coord - c_alpha_coord)
        t_i = (c_coord - c_alpha_coord) / np.linalg.norm(c_coord - c_alpha_coord)
        n_i = np.cross(u_i, t_i) / np.linalg.norm(np.cross(u_i, t_i))
        v_i = np.cross(n_i, u_i)
        assert (math.fabs(
            np.linalg.norm(v_i) - 1.) < 1e-5), "protein utils protein_to_graph_dips, v_i norm larger than 1"
        n_i_list.append(n_i)
        u_i_list.append(u_i)
        v_i_list.append(v_i)
        residue_representatives_loc_list.append(c_alpha_coord)

    residue_representatives_loc_feat = np.stack(residue_representatives_loc_list, axis=0)  # (N_res, 3)
    n_i_feat = np.stack(n_i_list, axis=0)
    u_i_feat = np.stack(u_i_list, axis=0)
    v_i_feat = np.stack(v_i_list, axis=0)
    num_residues = len(c_alpha_coords)
    if num_residues <= 1:
        raise ValueError(f"rec contains only 1 residue!")

    ################### Build the k-NN graph ##############################
    assert num_residues == residue_representatives_loc_feat.shape[0]
    assert residue_representatives_loc_feat.shape[1] == 3
    distances = spa.distance.cdist(c_alpha_coords, c_alpha_coords)
    src_list = []
    dst_list = []
    dist_list = []
    mean_norm_list = []
    for i in range(num_residues):
        dst = list(np.where(distances[i, :] < cutoff)[0])
        dst.remove(i)
        if max_neighbor != None and len(dst) > max_neighbor:
            dst = list(np.argsort(distances[i, :]))[1: max_neighbor + 1]
        if len(dst) == 0:
            dst = list(np.argsort(distances[i, :]))[1:2]  # choose second because first is i itself
            log(
                f'The c_alpha_cutoff {cutoff} was too small for one c_alpha such that it had no neighbors. So we connected it to the closest other c_alpha')
        assert i not in dst
        src = [i] * len(dst)
        src_list.extend(src)
        dst_list.extend(dst)
        valid_dist = list(distances[i, dst])
        dist_list.extend(valid_dist)
        valid_dist_np = distances[i, dst]
        sigma = np.array([1., 2., 5., 10., 30.]).reshape((-1, 1))
        weights = softmax(- valid_dist_np.reshape((1, -1)) ** 2 / sigma, axis=1)  # (sigma_num, neigh_num)
        assert weights[0].sum() > 1 - 1e-2 and weights[0].sum() < 1.01
        diff_vecs = residue_representatives_loc_feat[src, :] - residue_representatives_loc_feat[
                                                               dst, :]  # (neigh_num, 3)
        mean_vec = weights.dot(diff_vecs)  # (sigma_num, 3)
        denominator = weights.dot(np.linalg.norm(diff_vecs, axis=1))  # (sigma_num,)
        mean_vec_ratio_norm = np.linalg.norm(mean_vec, axis=1) / denominator  # (sigma_num,)
        mean_norm_list.append(mean_vec_ratio_norm)
    assert len(src_list) == len(dst_list)
    assert len(dist_list) == len(dst_list)
    graph = dgl.graph((torch.tensor(src_list), torch.tensor(dst_list)), num_nodes=num_residues, idtype=torch.int32)

    graph.ndata['feat'] = rec_residue_featurizer(rec)
    graph.edata['feat'] = distance_featurizer(dist_list, divisor=4)  # avg distance = 7. So divisor = (4/7)*7 = 4

    # Loop over all edges of the graph and build the various p_ij, q_ij, k_ij, t_ij pairs
    edge_feat_ori_list = []
    for i in range(len(dist_list)):
        src = src_list[i]
        dst = dst_list[i]
        # place n_i, u_i, v_i as lines in a 3x3 basis matrix
        basis_matrix = np.stack((n_i_feat[dst, :], u_i_feat[dst, :], v_i_feat[dst, :]), axis=0)
        p_ij = np.matmul(basis_matrix,
                         residue_representatives_loc_feat[src, :] - residue_representatives_loc_feat[
                                                                    dst, :])
        q_ij = np.matmul(basis_matrix, n_i_feat[src, :])  # shape (3,)
        k_ij = np.matmul(basis_matrix, u_i_feat[src, :])
        t_ij = np.matmul(basis_matrix, v_i_feat[src, :])
        s_ij = np.concatenate((p_ij, q_ij, k_ij, t_ij), axis=0)  # shape (12,)
        edge_feat_ori_list.append(s_ij)
    edge_feat_ori_feat = np.stack(edge_feat_ori_list, axis=0)  # shape (num_edges, 4, 3)
    edge_feat_ori_feat = torch.from_numpy(edge_feat_ori_feat.astype(np.float32))
    graph.edata['feat'] = torch.cat([graph.edata['feat'], edge_feat_ori_feat], axis=1)  # (num_edges, 17)

    residue_representatives_loc_feat = torch.from_numpy(residue_representatives_loc_feat.astype(np.float32))
    graph.ndata['x'] = residue_representatives_loc_feat
    graph.ndata['mu_r_norm'] = torch.from_numpy(np.array(mean_norm_list).astype(np.float32))
    return graph



In [ ]:
def get_rec_graph(rec, rec_coords, c_alpha_coords, n_coords, c_coords, use_rec_atoms, rec_radius,
                     surface_graph_cutoff, surface_mesh_cutoff, c_alpha_max_neighbors=None, surface_max_neighbors=None):
    

    return get_calpha_graph(rec, c_alpha_coords, n_coords, c_coords, rec_radius, c_alpha_max_neighbors)